In [26]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [27]:
X,y = datasets.load_iris(return_X_y=True)
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("y_train : ",len(y_train))
print("X_train : ",X_train.shape)
print("y_test : ", len(y_test))
print("X_test : ",X_test.shape)

y_train :  120
X_train :  (120, 4)
y_test :  30
X_test :  (30, 4)


In [28]:
#logistic parameters
# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}
# train the model
lr = LogisticRegression(**params)
lr.fit(X_train,y_train)

/opt/anaconda3/envs/ths/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888)

In [29]:
# Predict on the test set
y_pred = lr.predict(X_test)
# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
print("accuracy : ",accuracy)

accuracy :  1.0


In [30]:
# Set our tracking server uri for logging
mlflow_uri ="http://127.0.0.1:8080"
mlflow.set_experiment(mlflow_uri)

# # Create a new MLflow Experiment
mlflow.set_experiment("v1_getting_start_mlflow")

Traceback (most recent call last):
  File "/opt/anaconda3/envs/ths/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 327, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/opt/anaconda3/envs/ths/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 421, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/opt/anaconda3/envs/ths/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 1367, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/opt/anaconda3/envs/ths/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 1360, in _read_helper
    result = read_yaml(root, file_name)
  File "/opt/anaconda3/envs/ths/lib/python3.10/site-packages/mlflow/utils/file_utils.py", line 309, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' does not exist.")
mlflow.exceptions.MissingCon

<Experiment: artifact_location='file:///Users/tharhtet/Documents/github/ML-in-Prod-batch-1/8_Experiment_Tracking/mlruns/670037321960006845', creation_time=1726947021172, experiment_id='670037321960006845', last_update_time=1726947021172, lifecycle_stage='active', name='v1_getting_start_mlflow', tags={}>

In [31]:
# Start an MLflow run
with mlflow.start_run():
   # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

       # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train
    )


In [32]:
print("Model URI : ",model_info.model_uri)

Model URI :  runs:/a16b95b440dc4cc581ae972ba1bfbc00/iris_model


### Load and predict 

In [33]:
X,y = datasets.load_iris(return_X_y=True)
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("y_train : ",len(y_train))
print("X_train : ",X_train.shape)
print("y_test : ", len(y_test))
print("X_test : ",X_test.shape)

y_train :  120
X_train :  (120, 4)
y_test :  30
X_test :  (30, 4)


In [34]:

model_uri = "runs:/ab7a4afde023458d841cfb821c37cc39/iris_model"

# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_uri)

predictions = loaded_model.predict(X_test)



MlflowException: Run 'ab7a4afde023458d841cfb821c37cc39' not found

In [35]:
import numpy as np
print(np.sum(predictions==y_test)/len(y_test) * 100)

NameError: name 'predictions' is not defined